
**Install requirements**

In [ ]:
!pip3 install 'torch==1.4.0'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'
!pip3 install 'Pillow'

**Import libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import copy
from torch.utils.data import DataLoader
from torch.backends import cudnn
import torchvision
from torchvision import transforms
from torchvision.models import alexnet
from tqdm import tqdm
import numpy as np
import cv2 as cv

from torchvision.datasets import VisionDataset
from PIL import Image
import os
import os.path

**Define the Dataset class, i.e. Symbols** 

In [ ]:
LABELS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'div', 'equal', 'minus', 'mul', 'plus']

def get_squared_img(image_pil):
    # Convert PIL image -> OpenCV image (1. Convert into Numpy array, 2. Convert RGB to BGR)
    image = np.array(image_pil)
    image = image[:, :, ::-1].copy()

    # Fit the image in a squared area
    height = image.shape[0]
    width = image.shape[1]
    size = max(height, width)
    image_prepared = np.zeros((size, size, 3), dtype=np.uint8)
    image_prepared[:, :] = (255, 255, 255)
    y_start = int((size-height)/2)
    x_start = int((size-width)/2)
    y_end = y_start + height
    x_end = x_start + width
    image_prepared[y_start:y_end, x_start:x_end] = image

    # Convert OpenCV image -> PIL image 
    image_prepared = cv.cvtColor(image_prepared, cv.COLOR_BGR2RGB)
    image_pil = Image.fromarray(image_prepared) 

    return image_pil

def pil_loader(path):
    with open(path, 'rb') as f:
        # Read the image from the filesystem 
        img = Image.open(f)
        img = img.convert('RGB')
        squared_img = get_squared_img(img)
        return squared_img

class Symbols(VisionDataset):
    def __init__(self, root, split="train", transform=None):
        '''
        __init__ creates the class containing the dataset (it contains an array of (pil_image, label) pairs)
        '''
        super(Symbols, self).__init__(root, transform=transform)        

        self.root = root
        self.transform = transform
        self.data = [] 

        if split == "train":
            rootdir = root + "train/"
        elif split == "eval":
            rootdir = root + "eval/"

        for root, subdirs, files in os.walk(rootdir):
          #print('--\nroot = ' + root)
          #for subdir in subdirs:
              #print('\t- subdirectory ' + subdir)

          for filename in files:
              file_path = os.path.join(root, filename)
              #print('\t- file %s (full path: %s)' % (filename, file_path))

              # Get the class label from the directory name
              root_tmp = root
              label = root_tmp.replace(rootdir, "")

              #print('Adding the pair... ({}, {})'.format(filename, label))

              # Retrieve the index of the label of the current image
              label_index = LABELS.index(label)

              obj = (pil_loader(file_path), label_index)
              self.data.append(obj)

    def __getitem__(self, index):
        '''
        __getitem__ should access an element through its index
        Args:
            index (int): Index
        Returns:
            tuple: (pil_image, label) 
        '''

        image = self.data[index][0]
        label = self.data[index][1]

        # Applies preprocessing when accessing the image
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        '''
        __len__ returns the length of the dataset
        '''
        length = len(self.data)
        return length

    def retrieveArrayForDebug(self):
      return self.data
      
    def retrieveLabelsForDebug(self):
      return self.labels

**Set Arguments**

In [ ]:
DEVICE = 'cuda'      # 'cuda' or 'cpu'

NUM_CLASSES = 15

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default
GAMMA = 0.1          # Multiplicative factor for learning rate step-down
LOG_FREQUENCY = 10

LR = 1e-2            # Initial Learning Rate 

# NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
# NUM_EPOCHS = 40      # Variation of the total number of training epochs
NUM_EPOCHS = 50      # Other variation of the total number of training epochs

STEP_SIZE = 30       # How many epochs before decreasing learning rate (if using a step-down policy)

**Define Data Preprocessing**

In [ ]:
# # Define transforms for training phase (NO DATA AUGMENTATION)
# data_transform = transforms.Compose([transforms.Resize(224),          # Resizes short size of the PIL image to 224
#                                       transforms.ToTensor(),          # Turn PIL Image to torch.Tensor
#                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
# ])

# # Define transforms for training phase (DATA AUGMENTATION - RandomRotation(10))
# data_transform = transforms.Compose([transforms.Resize(224),          # Resizes short size of the PIL image to 224
#                                       transforms.RandomRotation(10),  # Rotate the image by a random angle
#                                       transforms.ToTensor(),          # Turn PIL Image to torch.Tensor
#                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
# ])

# Define transforms for training phase (DATA AUGMENTATION - RandomRotation(15))
data_transform = transforms.Compose([transforms.Resize(224),          # Resizes short size of the PIL image to 224
                                      transforms.RandomRotation(15),  # Rotate the image by a random angle
                                      transforms.ToTensor(),          # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])

# Define transforms for validation phase 
eval_transform = transforms.Compose([transforms.Resize(224),      # Resizes short size of the PIL image to 224
                                     transforms.ToTensor(),       # Turn PIL Image to torch.Tensor
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])

**Prepare Dataset**

In [ ]:
# Clone github repository with data
if not os.path.isdir('./IPCV-Camera-Calculator'):
  !git clone https://github.com/ilagioda/IPCV-Camera-Calculator.git

DATA_DIR = 'IPCV-Camera-Calculator/dataset/'

# Prepare Pytorch train/val datasets
train_dataset = Symbols(DATA_DIR, split="train", transform=data_transform) 
val_dataset = Symbols(DATA_DIR, split="eval", transform=data_transform)

# DEBUG Check dataset sizes
# print('Train Dataset: {}'.format(len(train_dataset)))
# print('Validation Dataset: {}'.format(len(val_dataset)))

# DEBUG
# arr = train_dataset.retrieveArrayForDebug()
# print(arr[600][1])
# lab = train_dataset.retrieveLabelsForDebug()
# print(lab)

**Prepare Dataloaders**

In [ ]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

**Prepare Network**

In [ ]:
# Load AlexNet model
net = alexnet()

# Change the last layer of AlexNet to output NUM_CLASSES classes
net.classifier[6] = nn.Linear(4096, NUM_CLASSES)    # nn.Linear in pytorch is a fully connected layer
                                                    # The convolutional layer is nn.Conv2d

**Prepare Training**

In [ ]:
# Define loss function (for classification, we use Cross Entropy)
criterion = nn.CrossEntropyLoss()

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: net.features.parameters()
# In this case, we optimize over all the parameters of AlexNet
parameters_to_optimize = net.parameters()

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Train on the training set and evaluate on the validation set**

In [ ]:
# By default, everything is loaded to CPU
net = net.to(DEVICE)      # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark     # Calling this optimizes runtime

# Initialize the best accuracy
best_acc = -1
best_epoch = -1


# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train()   # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs = net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients


  # Evaluation of the model on the validation set
  running_corrects = 0
  for images, labels in tqdm(val_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()


  # Calculate Accuracy
  accuracy = running_corrects / float(len(val_dataset))
  print('Validation Accuracy: {} %'.format(accuracy*100))

  # Check if a better accuracy has been found
  if accuracy > best_acc:

    # Update the accuracy and the epoch
    best_acc = accuracy
    best_epoch = epoch + 1

    # Save the parameters of the best model
    best_net = copy.deepcopy(net)

  # Step the scheduler
  scheduler.step() 

# Print the best accuracy on validation set
print("Best accuracy on validation set (found at epoch {}/{} and with initial LR={}): {} %".format(best_epoch, NUM_EPOCHS, LR, best_acc*100))

**Prepare the finetuning on the validation set**



In [ ]:
# Choose parameters to optimize
parameters_to_optimize = best_net.parameters()

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Finetuning the best model on the validation set**

In [ ]:
# By default, everything is loaded to CPU
best_net = best_net.to(DEVICE)   # this will bring the best network to GPU if DEVICE is cuda

cudnn.benchmark   # Calling this optimizes runtime

current_step = 0
# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  # Iterate over the dataset
  for images, labels in val_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    best_net.train()    # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad()       # Zero-ing the gradients

    # Forward pass to the network
    outputs = best_net(images)

    # Compute loss based on output and ground truth
    loss = criterion(outputs, labels)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  # Step the scheduler
  scheduler.step()

**Save the trained model**

In [ ]:
PATH = './NN.pth'
torch.save(best_net.state_dict(), PATH)